In [1]:
# Creating RL MLP for Tic Tac Toe

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import random
from IPython.display import display, clear_output
from time import sleep

import numpy as np
import time
import sys
from functools import reduce

from utils import show_field, Board
from agents.random import RandomAgent

In [2]:
# Setting up GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
class IntelligentAgent:
    def __init__(self, nn_filename=None, board_dim=3, hidden_size=64):
        self.board_dim = board_dim
        if nn_filename is None:
            self.brain = AgentNN(board_dim, hidden_size)
        else:
            self.brain = torch.load(nn_filename)

        self.optimizer = optim.AdamW(self.brain.parameters(), amsgrad=True)
        self.brain.to(device)
        self.brain.eval()

        self.remap_move = {}
        pos = 0
        for i in range(board_dim):
            for j in range(board_dim):
                self.remap_move[pos] = (i, j)
                pos += 1

    def get_move(self, board):
        with torch.no_grad():
            thought = self.brain(torch.FloatTensor(board.board.flatten()).to(device))
            #print(thought, file=sys.stderr)
            return self.remap_move[thought.to('cpu').argmax().item()]

    def get_move_batch(self, boards):
        thoughts = self.brain(torch.stack([torch.FloatTensor(b[0].board.flatten()) for b in boards]).to(device))
        return thoughts

    def save(self, file_name):
        self.brain.save(file_name)

    def train(self, moves, results):
        self.brain.train()
        actual_moves = self.get_move_batch(moves)
        z = torch.zeros(actual_moves.shape[0], 9)
        for i, j in zip(range(actual_moves.shape[0]), actual_moves.argmax(dim=1)):
            z[i,j] = results[i]
        loss = F.cross_entropy(actual_moves, z.to(device))
        loss.backward()
        self.optimizer.step()
        self.optimizer.zero_grad()
        self.brain.eval()

In [4]:
class AgentNN(nn.Module):
    def __init__(self, board_dim=3, hidden_size=64):
        super().__init__()
        self.board_dim = board_dim
        self.hidden_size = hidden_size

        self.input = nn.Linear(board_dim ** 2, hidden_size)
        self.hidden = nn.Linear(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, board_dim ** 2)

    def forward(self, x):
        x = self.input(x)
        x = F.relu(x)
        x = self.hidden(x)
        x = F.relu(x)
        return self.out(x)

    def save(self, file_name):
        torch.save(self, file_name)


In [5]:
def play_single_game(agent_x, agent_o, show=False, win_condition=None):
    # Play game and return moves in reversed order

    assert agent_x.board_dim == agent_o.board_dim, 'Agents have different dims'
    b = Board(agent_x.board_dim, win_condition)
    turn = 0
    player = [1, -1]
    agent = (agent_x, agent_o)
    moves = ([], [])
    while b.empty_cells > 0:
        if show:
            b.show()
            time.sleep(1)
        move = agent[turn].get_move(b)
        moves[turn].insert(0, (b.deepcopy(), move))
        result = b.make_move(move, player[turn])
        if result != 0:
            if show:
                b.show()
                remap = {1: 'x', -1: 'o'}
                print(f"Player '{remap[int(result)]}' wins!")
                time.sleep(1)
            return result, moves
        turn += 1
        turn %= 2
    if show:
        b.show()
        print(f'Draw!')
        time.sleep(1)
    return 0, moves

In [6]:
agent_o = RandomAgent(3)
agent_x = IntelligentAgent('data/test_nn_9')
play_single_game(agent_x, agent_o, True, 3)

    0   1   2   
    -   -   -   
0 | x | x | x |
    -   -   -   
1 |   | o |   |
    -   -   -   
2 | o |   |   |
    -   -   -   
Player 'x' wins!


(1.0,
 ([(<utils.Board at 0x7f86620bae50>, (0, 1)),
   (<utils.Board at 0x7f87084e5610>, (0, 0)),
   (<utils.Board at 0x7f86e44e5490>, (0, 2))],
  [(<utils.Board at 0x7f87085d2c90>, (2, 0)),
   (<utils.Board at 0x7f86625e1b10>, (1, 1))]))

In [8]:
START = 0
NUM_EPOCHS = 10
NUM_GAMES = 1000
BATCH_SIZE = 100
PRINT_TIME = False
    

start_time = time.perf_counter()
for epoch in range(START, START + NUM_EPOCHS):
    file_name = None
    if epoch > 0:
        file_name = f'data/test_nn_{epoch - 1}'
    agent = IntelligentAgent(nn_filename=file_name, hidden_size=2048)
    random_agent = RandomAgent(3)
    
    wins = 0
    draws = 0
    total_moves = []
    results = []
    start_batch_time = time.perf_counter()
    for i in range(NUM_GAMES):
        res, moves = play_single_game(agent, random_agent)
        if res == 1:
            wins += 1
        if res == 0:
            draws += 1

        # custom loss for draw
        if res == 0:
            res = 0.9

        total_moves += moves[0]
        results += [res] * len(moves[0])
        
        if len(total_moves) >= BATCH_SIZE:
            if PRINT_TIME:
                print(f'Total time: {time.perf_counter() - start_time}, Time for creating current batch: {time.perf_counter() - start_batch_time}')
                start_train_time = time.perf_counter()

            agent.train(total_moves, results)

            if PRINT_TIME:
                print(f'Total time: {time.perf_counter() - start_time}, Time for training current batch: {time.perf_counter() - start_train_time}')


            # clear tmp variables
            total_moves = []
            results = []

            if PRINT_TIME:
                start_batch_time = time.perf_counter()

    if len(total_moves) > 0:
        agent.train(total_moves, results)


    print(f'Total time: {time.perf_counter() - start_time}, Win rate {wins / NUM_GAMES} Draw rate {draws / NUM_GAMES} Win or draw rate {(draws + wins) / NUM_GAMES}')
    agent.save(f'data/test_nn_{epoch}')

Total time: 1.381072675999917, Win rate 0.0 Draw rate 0.0 Win or draw rate 0.0
Total time: 2.8823781369999324, Win rate 0.109 Draw rate 0.0 Win or draw rate 0.109
Total time: 4.436343383000008, Win rate 0.091 Draw rate 0.0 Win or draw rate 0.091
Total time: 6.063992369999937, Win rate 0.078 Draw rate 0.0 Win or draw rate 0.078
Total time: 7.569201131, Win rate 0.435 Draw rate 0.0 Win or draw rate 0.435
Total time: 9.279927431999909, Win rate 0.584 Draw rate 0.0 Win or draw rate 0.584
Total time: 10.849007207999989, Win rate 0.624 Draw rate 0.0 Win or draw rate 0.624
Total time: 12.483505585999978, Win rate 0.621 Draw rate 0.0 Win or draw rate 0.621
Total time: 14.125079701000004, Win rate 0.655 Draw rate 0.0 Win or draw rate 0.655
Total time: 15.754408292999983, Win rate 0.813 Draw rate 0.0 Win or draw rate 0.813


In [191]:
agent_o = RandomAgent(3)
agent_x = IntelligentAgent(nn_filename='test_nn_9')
wins = 0
games = 1000
for i in range(games):
    res, _ = play_single_game(agent_x, agent_o)
    if res == 1:
        wins += 1
print(f'Win rate {wins / games}')

Win rate 0.0


In [196]:
# agent_o = IntelligentAgent(nn_filename='test_nn_9')
agent_x = IntelligentAgent(nn_filename='test_nn_73')
agent_o = RandomAgent(3)
# agent_x = RandomAgent(3)
play_single_game(agent_x, agent_o, show=True)

    0   1   2   
    -   -   -   
0 |   |   | x |
    -   -   -   
1 | o | x | o |
    -   -   -   
2 | x |   |   |
    -   -   -   
Player 'x' wins!


(1.0,
 ([(<__main__.Board at 0x177ce43d0>, (1, 1)),
   (<__main__.Board at 0x293947a90>, (0, 2)),
   (<__main__.Board at 0x293946e00>, (2, 0))],
  [(<__main__.Board at 0x177ce6290>, (1, 0)),
   (<__main__.Board at 0x177ce40d0>, (1, 2))]))

In [ ]:
# Our NN should return board size output with probabilities of best move
# We need to collect all steps of our agent
# How do we backprop with this information?